<a href="https://colab.research.google.com/github/gabrielblins/palestra_gdg/blob/main/notebooks/GoogleExtended_MaritacaLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integração Maritaca - Langchain

Temos um LLM para chamar de nosso! Os desenvolvedores da https://www.maritaca.ai estão de parabéns pelo excelente trabalho desenvolvido.

O objetivo deste notebook é demonstrar como integrar o LLM ao Langchain considerando as chamadas a API que estão disponíveis no momento, que é a inferência apenas, não há possibilidade de obter os embeddings, mas acho que é uma questão de tempo até o time disponibilizar essa funcionalidade.

Vamos ao que interessa.

### Software base
Vamos instalar o langchain e algumas outras coisas

In [1]:
!pip install -U -q langchain unstructured pinecone-client openai tiktoken python-dotenv faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 85.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 73.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Passo 1 - Criar um custom LLM

Para mais informações sobre como o Langchain funciona, sugiro ler a documentação do produto. Aqui vamos focar na integração com o Maritaca.

De todo o modo, o primeiro passo é criar uma extensão da classe LLM e assim entregar ao ambiente do langchain um objeto que "sabe" fazer chamadas ao Maritaca. Segundo a documentação, é preciso implementar o método _call que recebe uma string, executa a chamada ao LLM e retorna outra string como saída.

Já que o nome do modelo é MariTalk, vamos usar esse nome para nossa classe especial.

In [31]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from requests.adapters import HTTPAdapter, Retry
from langchain.llms.base import LLM

import requests

class MaritalkLLM(LLM):

  @property
  def _llm_type(self) -> str:
    return "custom"

  def _call(
    self,
    prompt: str,
    stop: Optional[List[str]] = None,
    run_manager: Optional[CallbackManagerForLLMRun] = None,
  ) -> str:
    session = self.retry_request_session()
    payload = {
        "messages": [
          {
            "role": "human",
            "content": prompt
          }
      ],
      "model": 'Maritalk',
      "do_sample": True,
      "max_tokens": 2048,
      "temperature": 0.5,
      "top_p": 0.95,
      "repetition_penalty": 1,
      "use_chat_template": True,
      "stopping_tokens": ['string']
    }
    res = session.post('https://chat.maritaca.ai/api/chat/inference',
                       json=payload)

    if res.status_code == 200:
      return res.json()['answer']
    else:
      print(f'oops... status code {res.status_code}')
      return ''

  def retry_request_session(self, retries: Optional[int] = 3):
    # we setup retry strategy to retry on common errors
    retries = Retry(
        total=retries,
        backoff_factor=0.1,
        status_forcelist=[
            408,  # request timeout
            429,  # too many requests
            500,  # internal server error
            502,  # bad gateway
            503,  # service unavailable
            504   # gateway timeout
        ]
    )
    # we setup a session with the retry strategy
    session = requests.Session()
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

  @property
  def _identifying_params(self) -> Mapping[str, Any]:
    """Get the identifying parameters."""
    return {"n": 1}

### Explicação

A classe acima extende LLM da Langchain para que possamos implementar nossa lógica.

O método retry_request_session é um helper que vai resolver pequenos problemas de comunicação que podem acontecer e retomar a chamada fazendo 3 tentativas.

Em _call realizamos a chamada a API passando o prompt e os demais parâmetros.

A seguir, o teste!

In [4]:
llm = MaritalkLLM()

In [5]:
llm('Me explique como funciona o SUS')

'O SUS (Sistema Único de Saúde) é um sistema de saúde pública no Brasil que tem como objetivo garantir o acesso universal e igualitário à saúde para todos os brasileiros. Ele é financiado por recursos do governo federal, estadual e municipal, além de contribuições da população por meio de impostos.\n\nO SUS é composto por uma rede de serviços de saúde que inclui hospitais, postos de saúde, clínicas, laboratórios, entre outros. Ele é organizado em três níveis de atenção à saúde: a atenção básica, a atenção de média complexidade e a atenção de alta complexidade.\n\nA atenção básica é o primeiro contato da população com o sistema de saúde e é responsável por atender as necessidades de saúde mais comuns, como consultas médicas, exames preventivos e tratamento de doenças crônicas. Ela é realizada em postos de saúde e clínicas da família e é coordenada por equipes de saúde da família.\n\nA atenção de média complexidade é responsável por atender casos que exigem uma maior especialização e tec

O SUS (Sistema Único de Saúde) é um sistema de saúde pública no Brasil que tem como objetivo garantir o acesso universal e igualitário à saúde para todos os brasileiros. Ele é financiado por recursos do governo federal, estadual e municipal, além de contribuições da população por meio de impostos.

O SUS é composto por uma rede de serviços de saúde que inclui hospitais, postos de saúde, clínicas, laboratórios, entre outros. Ele é organizado em três níveis de atenção à saúde: a atenção básica, a atenção de média complexidade e a atenção de alta complexidade.

A atenção básica é o primeiro contato da população com o sistema de saúde e é responsável por atender as necessidades de saúde mais comuns, como consultas médicas, exames preventivos e tratamento de doenças crônicas. Ela é realizada em postos de saúde e clínicas da família e é coordenada por equipes de saúde da família.

A atenção de média complexidade é responsável por atender casos que exigem uma maior especialização e tecnologia, como exames de imagem, consultas com especialistas e cirurgias eletivas. Ela é realizada em hospitais e clínicas especializadas.

A atenção de alta complexidade é responsável por atender casos que exigem alta tecnologia e recursos especializados, como transplantes, tratamento de câncer e traumas graves. Ela é realizada em hospitais de referência e centros de excelência em saúde.

O SUS também tem programas de prevenção e controle de doenças, como o Programa Nacional de Imunizações, que oferece vacinas gratuitas para a população, e o Programa Nacional de DST/AIDS, que promove a prevenção e o tratamento da AIDS e outras doenças sexualmente transmissíveis.

Em resumo, o SUS é um sistema de saúde público que tem como objetivo garantir o acesso universal e igualitário à saúde para todos os brasileiros, por meio de uma rede de serviços de saúde organizada em três níveis de atenção à saúde: a atenção básica, a atenção de média complexidade e a atenção de alta complexidade. Ele também tem programas de prevenção e controle de doenças.

### Prompts mais elaborados

Agora que conseguimos integrar, vamos usar um prompt mais sofisticado e ver como o modelo se comporta. A idea a seguir é pedir ao modelo para entender um contexto, extrarir perguntas dele e nos entregar a resposta em um determinado padrão.

In [6]:
prompt = """
O trecho a seguir contem fatos sobre o assunto que ele trata. Sua missão é extrair dele 3 perguntas e organizar a resposta em formato JSON da seguinte maneira:
[{"pergunta1": "aqui vai a pergunta extraída"},{"pergunta2": "aqui vai a pergunta extraída"}].

Contexto:
A revolta começou como uma série de protestos contra a cobrança de impostos e a falta de autonomia política para a província do Rio Grande do Sul.
Em 1835, Bento Gonçalves liderou uma rebelião que resultou na proclamação da República Rio-Grandense, um governo independente que se opunha ao governo central brasileiro.
"""

llm(prompt)

'[\n{"pergunta1": "Quando começou a Revolução Farroupilha?"},\n{"pergunta2": "Quem liderou a Revolução Farroupilha?"},\n{"pergunta3": "O que a Revolução Farroupilha buscava?"}\n]'

# Integração com Pinecone

Para realizar uma integração completa usando exclusivamente o Maritaca, a API precisaria de um endpoint para captura dos embeddings, mas isso ainda não está disponível, então, a seguir iremos utilizar a OpenAI para criar as representações dos textos, enviar para o Pinecone e usar o Maritaca no final do fluxo. Funciona assim:

1. Capturamos os textos que queremos trabalhar, neste caso, a Portaria de Consolidação N 1 do Ministério da Saúde.

2. Vamos quebrar o texto em pequenas partes.

3. Interagir com a OpenAI e capturar os embeddings dos trechos criados.

4. Enviar esses trechos para o Pinecone.

Perceba que os passos acima são executados uma só vez ou sempre que houver necessidade de atualizar a estrutura. O resultado é uma base com diversos vetores que representam cada um dos trechos que criamos. O Pinecone, ou qualquer banco do mesmo tipo, viabiliza que sejam executadas consultas por similaridade. Dessa forma:

1. Agora que temos a base podemos enviar uma pergunta ou um texto qualquer, mas temos que criar a representação da pergunta e neste momento iremos chamar novamente a OpenAI para captura dos embeddings.

2. Em seguida, a Langchain interage com o Pinecone e pede para retornar os N vetores que mais se parecem com a pergunta/texto que enviamos.

Deste ponto em diante, já temos acesso aos N trechos semelhantes e, portanto, são os que possívelmente respondem a nossa pergunta. Vamos usar eles como contexto do prompt final com Maritaca.

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://repositorio.ufrn.br/bitstream/123456789/46616/1/AvaliaçãoAssociaçãoEletrooculográficos_Oliveira_2022.pdf")

In [24]:
data = loader.load()

In [25]:
print (f'Você tem {len(data)} documento(s) na base')
print (f'Há {len(data[0].page_content)} caracteres no documento')

Você tem 1 documento(s) na base
Há 58236 caracteres no documento


In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100, separators=['\n', ' ', ''])
texts = text_splitter.split_documents(data)

In [27]:
print (f'Agora há {len(texts)} documentos')

Agora há 152 documentos


In [28]:
texts[50].page_content

'1.3.1 Sacadas e microssacadas\n\nAs sacadas e microssacadas são tipos de movimentos oculares. As sacadas são\n\nmovimentos simultâneos de ambos os olhos em duas ou mais fases para fixação na mesma\n\ndireção (CASSIN; SOLOMON, 1990). As sacadas podem ser usadas para estudar o\n\ncomportamento ocular, os Movimentos Rápidos dos Olhos, tradicionalmente denominados\n\npela sigla em inglês de Rapid Eye Movements (REM) durante o estágio do sono, além de'

### Configuração da OpenAI e Pinecone

Recomendo ler como o Pinecone funciona, há muita documentação pela internet.

O código a seguir realiza a integração entre Pinecone e OpenAI e para isso você vai precisar de chaves. Para manter as que uso de forma privada, criei um arquivo .env com o seguinte formato:

```
    OPENAI_API_KEY=key
    PINECONE_API_KEY=key
    PINECONE_API_ENV=key
    PINECONE_INDEX=key
```

Chamei de .env e coloquei na raiz do colab. Você pode fazer o mesmo para testar este notebook.

In [29]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [32]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']
INDEX_NAME = os.environ['PINECONE_INDEX']

In [33]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [34]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = INDEX_NAME

In [ ]:
# Use o código a seguir para criar a base
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name, namespace=index_name)

# Use o código a seguir se já tiver um índice criado
# docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

### Configuração da FAISS vector store

In [35]:
import faiss
from langchain.vectorstores import FAISS
import pickle

def store_embeddings(docs, embeddings, sotre_name, path):

    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore


In [36]:
docsearch = FAISS.from_texts([t.page_content for t in texts], embeddings)

### Base construída

Agora temos nossa base vetorizada! Nosso PDF contendo os dados da Portaria de Consolidação já está no Pinecone, vamos testar!

A variável query vai armazenar o texto que será usado para a pesquisa por similaridade. Note que ao criar o objeto docsearch a variável embeddings foi passada como parâmetro. Ela será usada para criar uma representação da query e comparar com os dados contidos no Pinecone. Até o momento, a Maritaca não tem essa opção de criar embeddings, então vamos usar a OpenAI mesmo.

In [37]:
query = "O que é sacada?" # faça uma pergunta ou escreva um texto.
docs = docsearch.similarity_search(query)

In [38]:
docs

[Document(page_content='1.3.1 Sacadas e microssacadas\n\nAs sacadas e microssacadas são tipos de movimentos oculares. As sacadas são\n\nmovimentos simultâneos de ambos os olhos em duas ou mais fases para fixação na mesma\n\ndireção (CASSIN; SOLOMON, 1990). As sacadas podem ser usadas para estudar o\n\ncomportamento ocular, os Movimentos Rápidos dos Olhos, tradicionalmente denominados\n\npela sigla em inglês de Rapid Eye Movements (REM) durante o estágio do sono, além de', metadata={}),
 Document(page_content='consolidado no meio acadêmico, possuir diversas bibliotecas para análise, o laboratório já\n\n17\n\npossui equipamentos para esse tipo de análise, e, ainda, os dados que serão utilizados já foram\n\ncoletados usando EOG.\n\n1.3 Padrões oculares\n\nExistem alguns padrões oculares de interesse a serem analisados neste trabalho, entre\n\neles piscadas e sacadas, não serão analisados microssacadas e nem períodos de fixação.\n\n1.3.1 Sacadas e microssacadas', metadata={}),
 Document(pa

### Integração final

Agora vamos usar os textos vindos do Pinecone e pedir para nosso LLM, o MariTalk, criar uma resposta adequada.

In [39]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

PROMPT = """
Use os trechos a seguir para responder a pergunta no final. Se você não souber a resposta, apenas diga que não sabe, não tente criar uma resposta.

{context}

Pergunta: {question}
Resposta:
"""

chain_prompt = PromptTemplate(input_variables=["context", "question"], template=PROMPT)

# note a variável llm sendo passada abaixo, ela é o Maritaca
chain = load_qa_chain(llm, chain_type="stuff", verbose=True, prompt=chain_prompt)

In [40]:
query = "O que são sacadas?" # faça uma pergunta ou escreva um texto.

# no exemplo abaixo reduzimos o número de documentos para caber na janela do
# Maritaca
docs = docsearch.similarity_search(query, k=5)
chain.run(input_documents=docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use os trechos a seguir para responder a pergunta no final. Se você não souber a resposta, apenas diga que não sabe, não tente criar uma resposta.

1.3.1 Sacadas e microssacadas

As sacadas e microssacadas são tipos de movimentos oculares. As sacadas são

movimentos simultâneos de ambos os olhos em duas ou mais fases para fixação na mesma

direção (CASSIN; SOLOMON, 1990). As sacadas podem ser usadas para estudar o

comportamento ocular, os Movimentos Rápidos dos Olhos, tradicionalmente denominados

pela sigla em inglês de Rapid Eye Movements (REM) durante o estágio do sono, além de

consolidado no meio acadêmico, possuir diversas bibliotecas para análise, o laboratório já

17

possui equipamentos para esse tipo de análise, e, ainda, os dados que serão utilizados já foram

coletados usando EOG.

1.3 Padrões oculares

Existem alguns padrões oculares de interesse a serem analisados n

'As sacadas são movimentos simultâneos de ambos os olhos em duas ou mais fases para fixação na mesma direção. Elas podem ser usadas para estudar o comportamento ocular, os Movimentos Rápidos dos Olhos (REM) durante o estágio do sono, entre outras coisas.'

As sacadas são movimentos simultâneos de ambos os olhos em duas ou mais fases para fixação na mesma direção. Elas podem ser usadas para estudar o comportamento ocular, os Movimentos Rápidos dos Olhos (REM) durante o estágio do sono, entre outras coisas.